# 第13章: GBDT入門 - LightGBM & XGBoost

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] LightGBMとXGBoostのScikit-learnラッパーが使える
- [ ] Early Stoppingで過学習を防げる
- [ ] 学習曲線を可視化してモデルの挙動を理解できる
- [ ] Scikit-learnのモデルとGBDTを比較できる

## 🎯 前提知識

この章を学ぶには以下の知識が必要です：

- ✅ Notebook 05（決定木とアンサンブル学習）
- ✅ Notebook 03（評価指標とクロスバリデーション）
- ✅ Notebook 02（前処理）

⏱️ **推定学習時間**: 90分  
📊 **難易度**: ★★★★☆（上級）  
🎓 **カテゴリ**: 機械学習・GBDT

---

## 🚀 GBDTとは？

**GBDT（Gradient Boosting Decision Tree）** は、Kaggleや実務で最も頻繁に使われる機械学習アルゴリズムです。

**主な特徴：**
- 決定木を順次学習させて精度を高める（Boosting）
- テーブルデータ（表形式データ）で最強クラスの性能
- 特徴量エンジニアリングとの相性が良い

**代表的なライブラリ：**
1. **LightGBM** (Microsoft): 高速・軽量・メモリ効率が良い
2. **XGBoost**: 歴史が長く、安定した性能
3. **CatBoost** (Yandex): カテゴリ変数の扱いに優れる（次章で学習）

---

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# LightGBM and XGBoost
try:
    import lightgbm as lgb
    from lightgbm import LGBMClassifier
    print("✅ LightGBM installed")
except ImportError:
    print("⚠️ LightGBM not installed. Run: pip install lightgbm")

try:
    import xgboost as xgb
    from xgboost import XGBClassifier
    print("✅ XGBoost installed")
except ImportError:
    print("⚠️ XGBoost not installed. Run: pip install xgboost")

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

#### 📦 必要なライブラリの読み込み

このセクションでは、分析に必要なPythonライブラリを読み込みます。

**主要ライブラリの役割**：
- **NumPy**: 数値計算の基盤（配列操作、数学関数）
- **Pandas**: データ操作と分析（表形式データの処理）
- **Matplotlib / Seaborn**: データ可視化（グラフ作成）
- **scikit-learn**: 機械学習アルゴリズムとツール
- **LightGBM**: Microsoft開発の高速GBDTライブラリ
- **XGBoost**: 高性能GBDTライブラリ

**注意**: LightGBMとXGBoostがインストールされていない場合は、以下のコマンドでインストールしてください：
```bash
pip install lightgbm xgboost
```

## Part 1: Data Loading and Basic Comparison

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Load breast cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Create DataFrame for easier analysis
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y

print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"\nTarget distribution:")
print(df['target'].value_counts())
print(f"\nFeature names (first 10):")
print(list(data.feature_names[:10]))

**データセットの説明**：

このデータセットは、乳がんの診断データです：
- **サンプル数**: 569件
- **特徴量数**: 30個（腫瘍の形状、大きさなどの測定値）
- **目的変数**: 良性（0）か悪性（1）か

このデータセットでGBDTの基本的な使い方を学びます。

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nTraining class distribution:")
print(pd.Series(y_train).value_counts())

**分割後のデータ**：
- `X_train`, `y_train`: モデルの学習に使用
- `X_test`, `y_test`: モデルの評価に使用

`stratify=y`パラメータで、訓練セットとテストセットで
クラス比率を同じに保ちます。

## Part 2: Scikit-learn API - 同じ使い方で使える！

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Train three models with the same API
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, verbose=-1),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
}

results = []

for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    
    # Evaluate
    train_acc = model.score(X_train, y_train)
    test_acc = accuracy_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc
    })

# Display results
df_results = pd.DataFrame(results)
print("\n" + "="*60)
print("Model Comparison: RandomForest vs LightGBM vs XGBoost")
print("="*60)
print(df_results.to_string(index=False))

**重要なポイント**：

LightGBMとXGBoostは、**Scikit-learnと全く同じAPI**を使えます：
- `model.fit(X_train, y_train)` で学習
- `model.predict(X_test)` で予測
- `model.score(X, y)` で評価

既存のScikit-learnコードをそのまま置き換えるだけで、
高性能なGBDTモデルが使えます！

**精度の違い**：
- RandomForestも十分高精度
- LightGBMとXGBoostはさらに高精度になることが多い
- 特にデータが大きい場合、GBDTの優位性が顕著

In [ ]:
# Visualize comparison
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(df_results))
width = 0.35

ax.bar(x - width/2, df_results['Train Accuracy'], width, label='Train', alpha=0.8)
ax.bar(x + width/2, df_results['Test Accuracy'], width, label='Test', alpha=0.8)

ax.set_ylabel('Accuracy')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(df_results['Model'])
ax.legend()
ax.set_ylim([0.9, 1.0])
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**可視化の重要性**：
データや結果をグラフで表示すると、以下のことが分かります：
- モデル間の性能差
- 訓練精度とテスト精度のギャップ（過学習の度合い）
- どのモデルが最も安定しているか

「百聞は一見にしかず」です。数字だけでなく、
グラフで見ると直感的に理解できます。

## Part 3: Early Stopping - 過学習を自動で防ぐ！

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Split validation set from training set
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print("Dataset split:")
print(f"  Training: {len(X_train_sub)} samples")
print(f"  Validation: {len(X_val)} samples")
print(f"  Test: {len(X_test)} samples")

**Early Stoppingとは？**

Early Stoppingは、過学習を防ぐ強力なテクニックです：

1. 学習中に検証データ（Validation set）で精度を監視
2. 検証精度が改善しなくなったら、学習を自動で停止
3. 最も良かった時点のモデルを採用

**メリット**：
- 過学習を防げる
- 無駄な学習時間を削減できる
- ハイパーパラメータ調整が楽になる

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# LightGBM with early stopping
lgbm_model = LGBMClassifier(
    n_estimators=1000,  # 大きめに設定（早期終了するので問題ない）
    random_state=42,
    verbose=-1
)

# Fit with early stopping
lgbm_model.fit(
    X_train_sub, y_train_sub,
    eval_set=[(X_val, y_val)],
    eval_metric='logloss',
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
)

print(f"\nLightGBM Early Stopping Results:")
print(f"  Best iteration: {lgbm_model.best_iteration_}")
print(f"  Total iterations set: {lgbm_model.n_estimators}")
print(f"  Stopped early at: {lgbm_model.best_iteration_} trees")
print(f"\n  Test Accuracy: {lgbm_model.score(X_test, y_test):.4f}")

**Early Stoppingの結果**：

- `n_estimators=1000` に設定したが、実際には途中で停止
- `stopping_rounds=50` は「50回改善しなかったら停止」という意味
- 最適な木の数が自動で見つかる

これにより、過学習を防ぎつつ、最高の性能を引き出せます。

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# XGBoost with early stopping
xgb_model = XGBClassifier(
    n_estimators=1000,
    random_state=42,
    eval_metric='logloss'
)

xgb_model.fit(
    X_train_sub, y_train_sub,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=50,
    verbose=False
)

print(f"\nXGBoost Early Stopping Results:")
print(f"  Best iteration: {xgb_model.best_iteration}")
print(f"  Total iterations set: {xgb_model.n_estimators}")
print(f"  Stopped early at: {xgb_model.best_iteration} trees")
print(f"\n  Test Accuracy: {xgb_model.score(X_test, y_test):.4f}")

**LightGBM vs XGBoostのEarly Stopping**：

どちらも同じ概念ですが、書き方が少し違います：

- **LightGBM**: `callbacks=[lgb.early_stopping(...)]`
- **XGBoost**: `early_stopping_rounds=...`

どちらも`eval_set`に検証データを渡すことで動作します。

## Part 4: Learning Curves - 学習の推移を可視化

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Train models and track performance
n_trees_range = range(10, 501, 10)

lgbm_train_scores = []
lgbm_test_scores = []
xgb_train_scores = []
xgb_test_scores = []

for n_trees in n_trees_range:
    # LightGBM
    lgbm = LGBMClassifier(n_estimators=n_trees, random_state=42, verbose=-1)
    lgbm.fit(X_train, y_train)
    lgbm_train_scores.append(lgbm.score(X_train, y_train))
    lgbm_test_scores.append(lgbm.score(X_test, y_test))
    
    # XGBoost
    xgb_clf = XGBClassifier(n_estimators=n_trees, random_state=42, eval_metric='logloss')
    xgb_clf.fit(X_train, y_train)
    xgb_train_scores.append(xgb_clf.score(X_train, y_train))
    xgb_test_scores.append(xgb_clf.score(X_test, y_test))

print("Learning curve data collected.")

In [ ]:
# Plot learning curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# LightGBM
axes[0].plot(n_trees_range, lgbm_train_scores, 'b-', label='Train', linewidth=2)
axes[0].plot(n_trees_range, lgbm_test_scores, 'r-', label='Test', linewidth=2)
axes[0].set_xlabel('Number of Trees')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('LightGBM Learning Curve')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# XGBoost
axes[1].plot(n_trees_range, xgb_train_scores, 'b-', label='Train', linewidth=2)
axes[1].plot(n_trees_range, xgb_test_scores, 'r-', label='Test', linewidth=2)
axes[1].set_xlabel('Number of Trees')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('XGBoost Learning Curve')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**学習曲線の読み方**：

1. **訓練精度（青線）**:
   - 木の数が増えるほど上昇
   - 最終的に1.0（完璧）に近づく

2. **テスト精度（赤線）**:
   - 最初は上昇するが、途中で頭打ちか低下
   - 低下し始めたら過学習のサイン

3. **最適な木の数**:
   - テスト精度が最大になる点
   - Early Stoppingで自動的に見つけられる

**LightGBM vs XGBoost**:
- どちらも似た挙動を示す
- データセットによってどちらが良いかは異なる
- 両方試して比較するのがベストプラクティス

## Part 5: Speed Comparison - 速度を比較

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

import time

# Larger dataset for speed comparison
from sklearn.datasets import make_classification

X_large, y_large = make_classification(
    n_samples=10000,
    n_features=50,
    n_informative=30,
    n_classes=2,
    random_state=42
)

X_train_large, X_test_large, y_train_large, y_test_large = train_test_split(
    X_large, y_large, test_size=0.2, random_state=42
)

print("Large dataset created:")
print(f"  Training: {len(X_train_large)} samples")
print(f"  Features: {X_large.shape[1]}")

In [ ]:
# Compare training speed
models_speed = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, verbose=-1),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
}

speed_results = []

for name, model in models_speed.items():
    start_time = time.time()
    model.fit(X_train_large, y_train_large)
    elapsed_time = time.time() - start_time
    
    test_acc = model.score(X_test_large, y_test_large)
    
    speed_results.append({
        'Model': name,
        'Training Time (s)': elapsed_time,
        'Test Accuracy': test_acc
    })

df_speed = pd.DataFrame(speed_results)
print("\n" + "="*60)
print("Speed Comparison (10,000 samples, 50 features)")
print("="*60)
print(df_speed.to_string(index=False))

In [ ]:
# Visualize speed comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training time
axes[0].bar(df_speed['Model'], df_speed['Training Time (s)'], color=['#3498db', '#2ecc71', '#e74c3c'])
axes[0].set_ylabel('Training Time (seconds)')
axes[0].set_title('Training Speed Comparison')
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].bar(df_speed['Model'], df_speed['Test Accuracy'], color=['#3498db', '#2ecc71', '#e74c3c'])
axes[1].set_ylabel('Test Accuracy')
axes[1].set_title('Model Accuracy Comparison')
axes[1].set_ylim([0.8, 1.0])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**速度比較の結果**：

一般的な傾向：
1. **LightGBM**: 最も高速（数倍速いことも）
2. **XGBoost**: 中程度の速度
3. **RandomForest**: 相対的に遅い（特にデータが大きい場合）

**重要なポイント**：
- LightGBMは「軽量」の名の通り、非常に高速
- データサイズが大きいほど、速度差が顕著
- 精度はどれも高い水準で、速度とのトレードオフを考える

**実務では**：
- プロトタイピング: LightGBM（高速実験）
- 本番環境: 速度と精度のバランスで選択
- Kaggle: 両方試してアンサンブル

## Part 6: Feature Importance - 重要な特徴量を知る

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Train models on original dataset
lgbm_final = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
lgbm_final.fit(X_train, y_train)

xgb_final = XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
xgb_final.fit(X_train, y_train)

# Get feature importances
lgbm_importances = pd.DataFrame({
    'feature': data.feature_names,
    'importance': lgbm_final.feature_importances_
}).sort_values('importance', ascending=False)

xgb_importances = pd.DataFrame({
    'feature': data.feature_names,
    'importance': xgb_final.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 10 Important Features (LightGBM):")
print(lgbm_importances.head(10).to_string(index=False))

In [ ]:
# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# LightGBM
top_n = 15
axes[0].barh(range(top_n), lgbm_importances['importance'].head(top_n))
axes[0].set_yticks(range(top_n))
axes[0].set_yticklabels(lgbm_importances['feature'].head(top_n))
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance')
axes[0].set_title('LightGBM Feature Importance (Top 15)')
axes[0].grid(True, alpha=0.3)

# XGBoost
axes[1].barh(range(top_n), xgb_importances['importance'].head(top_n))
axes[1].set_yticks(range(top_n))
axes[1].set_yticklabels(xgb_importances['feature'].head(top_n))
axes[1].invert_yaxis()
axes[1].set_xlabel('Importance')
axes[1].set_title('XGBoost Feature Importance (Top 15)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Feature Importanceの活用法**：

1. **モデルの解釈**:
   - どの特徴量が予測に重要か理解できる
   - ビジネス上の意思決定に役立つ

2. **特徴量選択**:
   - 重要度の低い特徴量を削除してモデルを簡素化
   - 過学習を防ぐ
   - 推論速度を向上

3. **新しい特徴量の発見**:
   - 重要な特徴量から派生特徴量を作成
   - より良いモデルへの手がかり

**LightGBM vs XGBoost**:
- 重要度の計算方法が若干異なる
- 上位の特徴量はだいたい一致することが多い
- 両方見て総合的に判断するのが良い

## Part 7: Cross-Validation with GBDT

In [ ]:
# ============================================================
# [コードの説明]
# ============================================================

# Cross-validation for all models
from sklearn.model_selection import cross_val_score

models_cv = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=42, verbose=-1),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
}

print("\n" + "="*60)
print("Cross-Validation Results (5-Fold)")
print("="*60)

cv_results = []

for name, model in models_cv.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    
    cv_results.append({
        'Model': name,
        'Mean CV Score': scores.mean(),
        'Std CV Score': scores.std(),
        'Min Score': scores.min(),
        'Max Score': scores.max()
    })
    
    print(f"\n{name}:")
    print(f"  CV Scores: {scores}")
    print(f"  Mean: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

df_cv = pd.DataFrame(cv_results)
print("\n" + df_cv.to_string(index=False))

**Cross-Validationの重要性**：

単一のテストセットでの評価は、運が良いだけかもしれません。
Cross-Validationで複数の分割パターンで評価することで、
**本当の実力**を測定できます。

**結果の見方**：
- **Mean CV Score**: 平均精度（高いほど良い）
- **Std CV Score**: 標準偏差（小さいほど安定）
- **Min/Max Score**: 最悪/最良のケース

実務では、平均だけでなく標準偏差も重要です。
標準偏差が大きいモデルは、データによって性能が不安定です。

## Summary

Congratulations! You've learned the fundamentals of GBDT.

### What You've Learned

**基本的な使い方**
- LightGBMとXGBoostは、Scikit-learnと同じAPIで使える
- `fit()`, `predict()`, `score()` が全て使える
- 既存コードの置き換えが簡単

**Early Stopping**
- 過学習を自動で防ぐ強力な機能
- 検証データ（eval_set）を渡すだけで動作
- 最適な木の数を自動で見つける

**学習曲線**
- モデルの挙動を可視化して理解
- 過学習の兆候を発見
- 適切な木の数を決定

**速度と性能**
- LightGBMは非常に高速
- XGBoostは安定した性能
- RandomForestより高精度なことが多い

**Feature Importance**
- 重要な特徴量を特定
- モデルの解釈に役立つ
- 特徴量エンジニアリングの指針

### Next Steps
Continue to **Notebook 14** for CatBoost and categorical feature handling!

### ⚠️ よくあるエラー #1: LightGBMのインストール失敗

LightGBMのインストールで"Microsoft Visual C++が必要"エラーが出ることがあります。

**原因:**
1. Windows環境でC++コンパイラがインストールされていない
2. ビルド済みバイナリが見つからない

**✅ 解決法:**

```bash
# Windows: condaを使う方法（推奨）
conda install -c conda-forge lightgbm

# またはpre-builtホイールを使う
pip install lightgbm --prefer-binary

# macOS/Linux: 通常のpipで問題なし
pip install lightgbm
```

---

### ⚠️ よくあるエラー #2: Early Stoppingが効かない

Early Stoppingを設定したのに、全ての木が学習されてしまう。

**原因:**
1. `eval_set`を渡していない
2. `eval_metric`が正しく設定されていない

**✅ 解決法:**

```python
# ❌ 間違い: eval_setがない
model.fit(X_train, y_train)

# ✅ 正しい: eval_setを渡す
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)
```

---

---

## 🎓 自己評価クイズ

学習内容を確認しましょう！すぐに答えを見ずに、まず自分で考えてみてください。

### Q1: LightGBMとXGBoostの最大の利点は何ですか？

<details>
<summary>💡 答えを見る</summary>

**答え**: Scikit-learnと同じAPIで、より高い精度と速度が得られること

LightGBMとXGBoostは、Scikit-learnの`RandomForestClassifier`などと全く同じ使い方ができます。そのため、既存のコードを簡単に置き換えられます。さらに、テーブルデータでは通常、より高い精度を達成でき、特にLightGBMは非常に高速です。

</details>

---

### Q2: Early Stoppingの役割は何ですか？

<details>
<summary>💡 答えを見る</summary>

**答え**: 過学習を防ぎ、最適な木の数を自動で見つける

Early Stoppingは、検証データの性能を監視しながら学習を進め、性能が改善しなくなったら自動で学習を停止します。これにより、過学習を防ぎつつ、最適なモデルを得られます。また、無駄な計算時間も節約できます。

</details>

---

### Q3: Feature Importanceを見ることで何が分かりますか？

<details>
<summary>💡 答えを見る</summary>

**答え**: どの特徴量が予測に重要か、モデルの解釈ができる

Feature Importanceは、各特徴量が予測にどれだけ貢献しているかを示します。これにより、モデルの解釈が可能になり、重要でない特徴量を削除したり、重要な特徴量から新しい派生特徴量を作成したりできます。また、ビジネス上の意思決定にも役立ちます。

</details>

---

</details>

---

## ➡️ 次のステップ

### 学習を続ける

次は **Notebook 14: CatBoostとカテゴリ変数の攻略** へ進みましょう！

CatBoostは、カテゴリ変数（"male"/"female"のような文字列データ）を
OneHotEncodingなしで直接扱える革新的なライブラリです。

### 復習が必要な場合

- **Notebook 05: Tree and Ensemble Models**
- **Notebook 03: Model Evaluation Metrics**

### さらに学ぶために

**公式ドキュメント:**
- LightGBM: https://lightgbm.readthedocs.io/
- XGBoost: https://xgboost.readthedocs.io/

**書籍:**
- "Hands-On Gradient Boosting with XGBoost and scikit-learn"

**Kaggleで実践:**
- Titanic Competition
- House Prices Competition

---

### 🎉 お疲れ様でした！

次の章でさらに深く探求しましょう！